# 123. Best Time to Buy and Sell Stock III

In [21]:
import sys
# print sys.getrecursionlimit()
sys.setrecursionlimit(99999)

In [22]:
### Dedupe prices [5,2,2,2,4,2]->[5,2,4,2]
### Romove monotone decreasing sequence [5,4,3,2,4]->[5,2]
### store calculated subproblem: profit, min price, max price
### profit[i,j] <= profit[i-1,j], so if profit[i-1,j]==0, then profit[i,j]=0
### profit[i,j] <= profit[i,j+1], so if profit[i,j+1]==0, then profit[i,j]=0
### the buy price must be local minimum.


class Solution(object):
    
    def _maxProfitOneTrans(self, prices, start, end, profit_dict, min_price_idx_dict, max_price_idx_dict):
        if profit_dict.has_key((start,end)):
            return profit_dict[(start,end)]
        elif profit_dict.get((start-1,end), None)==0:
            profit_dict[(start,end)] = 0
            return 0
        elif profit_dict.get((start,end+1), None)==0:
            profit_dict[(start,end)] = 0
            return 0
        
        if len(prices[start:end])==0:
            profit_dict[(start,end)] = 0
            return 0
        
        if len(prices[start:end])==1:
            profit_dict[(start,end)] = 0
            if prices[start]>prices[end-1]:
                min_price_idx_dict[(start,end)] = end-1
            else:
                min_price_idx_dict[(start,end)] = start
            return 0
        
        if min_price_idx_dict.has_key((start,end-1)):
            if prices[end-1]<prices[min_price_idx_dict[(start,end-1)]]:
                min_price_idx = end-1
            else:
                min_price_idx = min_price_idx_dict[(start,end-1)]
        elif min_price_idx_dict.has_key((start-1,end)):
            if min_price_idx_dict[(start-1,end)]==start-1:                
                min_price, min_price_idx_interval \
                    = min((price, day) for (day, price) in enumerate(prices[start:end]))
                min_price_idx = min_price_idx_interval + start
            else:
                min_price_idx = min_price_idx_dict[(start-1,end)]
        else:
            min_price, min_price_idx_interval \
                = min((price, day) for (day, price) in enumerate(prices[start:end]))
            min_price_idx = min_price_idx_interval + start
        
        min_price_idx_dict[(start,end)] = min_price_idx
        min_price = prices[min_price_idx]
        
        if max_price_idx_dict.has_key((start+1,end-1)):
            if prices[end-1]>prices[max_price_idx_dict[(start+1,end-1)]]:
                max_price_idx = end-1
            else:
                max_price_idx = max_price_idx_dict[(start+1,end-1)]
        elif max_price_idx_dict.has_key((start,end)):
            if max_price_idx_dict[(start,end)]==start:
                max_price, max_price_idx_interval \
                    = max((price, day) for (day, price) in enumerate(prices[start+1:end]))
                max_price_idx = max_price_idx_interval + start + 1
            else:
                max_price_idx = max_price_idx_dict[(start,end)]
        else:
            max_price, max_price_idx_interval \
                = max((price, day) for (day, price) in enumerate(prices[start+1:end]))
            max_price_idx = max_price_idx_interval + start + 1
        
        max_price_idx_dict[(start+1,end)] = max_price_idx
        max_price = prices[max_price_idx]

        if min_price_idx==start:
            return max_price-min_price
        
        profit_left = self._maxProfitOneTrans(prices,start, min_price_idx, 
                           profit_dict, min_price_idx_dict, max_price_idx_dict)
        profit_right = self._maxProfitOneTrans(prices,min_price_idx, end, 
                           profit_dict, min_price_idx_dict, max_price_idx_dict)
        profit = max(profit_left, profit_right,0)
        
        profit_dict[(start,end)] = profit
        return profit
    
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        profit_dict = {}
        min_price_idx_dict = {}
        max_price_idx_dict = {}
        
        for i in range(n):                
            profit = self._maxProfitOneTrans(prices,0,i,profit_dict,
                                             min_price_idx_dict, max_price_idx_dict) \
                   + self._maxProfitOneTrans(prices,i,n,profit_dict,
                                             min_price_idx_dict, max_price_idx_dict)
            if profit>max_profit:
                max_profit=profit
        return max_profit

s1 = Solution()   

In [23]:
### Time Limit Exceeded ###
class Solution(object):
    def _maxProfitOneTrans(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n = len(prices)        
        if n==0:
            return 0    
        
        buy_price = prices[0]
        sell_price = prices[0]
        min_price = prices[0]
        max_profit = 0
                
        for i in range(1,n):
            if prices[i]>sell_price:
                sell_price = prices[i]
            
            if prices[i]<min_price:
                min_price = prices[i]
            
            temp_buy_price = min(prices[i-1],min_price) 
            profit = prices[i]-temp_buy_price
            if profit>max_profit:
                max_profit = profit
                sell_price = prices[i]
                buy_price = temp_buy_price
        return (sell_price - buy_price)
    
    def maxProfit(self, prices):
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        for i in range(n):
            profit = self._maxProfitOneTrans(prices[:i]) \
                   + self._maxProfitOneTrans(prices[i:])
            if profit>max_profit:
                max_profit=profit       
        return max_profit

s2 = Solution()   

In [24]:
class Solution(object):
    def _maxProfitOneTrans(self, prices):
        n = len(prices)        
        if n<=1:
            return 0    
        
        min_price, min_price_day = min((price, day) for (day, price) in enumerate(prices))
        if min_price_day==0:
            return max(prices[1:])-min_price
        
        profit_left = self._maxProfitOneTrans(prices[: min_price_day])
        profit_right = self._maxProfitOneTrans(prices[min_price_day:])
        profit = max(profit_left, profit_right,0)
        return profit
    
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        for i in range(n):
            profit = self._maxProfitOneTrans(prices[:i]) \
                   + self._maxProfitOneTrans(prices[i:])
            if profit>max_profit:
                max_profit=profit
        return max_profit

s3 = Solution()   

In [25]:

class Solution(object):
    def _maxProfitOneTrans(self, prices):
        n = len(prices)        
        if n<=1:
            return 0    
        
        profit_buy_at_time0 = max(max(prices[1:])-prices[0], 0)
        profit = max(self._maxProfitOneTrans(prices[1:]), profit_buy_at_time0)
        return profit
    
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        for i in range(n):
            profit = self._maxProfitOneTrans(prices[:i]) \
                   + self._maxProfitOneTrans(prices[i:])
            if profit>max_profit:
                max_profit=profit       
        return max_profit

s4 = Solution()

In [26]:
from operator import itemgetter

class BTNode(object):
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right


class BTree(object):
    def __init__(self, root): 
        self.root = root;
          
    def insert(self, data):
        self.insertNode(data, self.root)
          
    def insertNode(self, data, btnode):
        if btnode == None:
            btnode = BTNode(data)
        elif data < btnode.data:
            if btnode.left == None:
                btnode.left = BTNode(data)
                return
              
            self.insertNode(data,btnode.left)
        elif data > btnode.data:
            if btnode.right == None:
                btnode.right = BTNode(data)
                return
            
            self.insertNode(data,btnode.right)
    
class Solution(object):
    def _maxProfitOneTransInterval(self, prices, btnode, interval_start_day):
        n = len(prices)        
        if n<=1:
            return 0    
        
        min_price_day = btnode.data - interval_start_day
        # print min_price_day, btnode.data, interval_start_day
        if min_price_day==0:
            return max(prices[1:])-prices[0]
        
        if btnode.left is not None:
            profit_left = self._maxProfitOneTransInterval(
                prices[: min_price_day], btnode.left, interval_start_day)
        else:
            profit_left = 0
        if btnode.right is not None:
            profit_right = self._maxProfitOneTransInterval(
                prices[min_price_day:], btnode, btnode.data)
        else:
            profit_right = 0
            
        profit = max(profit_left, profit_right,0)
        return profit

    
    def _maxProfitOneTrans(self, prices):
        min_price_index_tree = self._minPriceTree(prices)
        if min_price_index_tree is not None:
            return self._maxProfitOneTransInterval(prices, min_price_index_tree.root, 0)
        else:
            return 0 
    
    def _minPriceTree(self, prices):
        min_price_days = [i[0] for i in sorted(enumerate(prices), key=itemgetter(1))]
        if len(min_price_days)==0:
            return None
        root = BTNode(min_price_days[0])
        btree = BTree(root)
        [btree.insert(i) for i in min_price_days[1:]]
        return btree
    
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        for i in range(n):
            trans1_profit = self._maxProfitOneTrans(prices[:i])
            trans2_profit = self._maxProfitOneTrans(prices[i:])
            profit = trans1_profit + trans2_profit
            if profit>max_profit:
                max_profit=profit

        return max_profit

s5 = Solution()   

In [27]:
class Solution(object):
    
    def _maxProfitOneTrans(self, prices, start, end, profit_dict, max_sell_price_dict):
        if profit_dict.has_key((start,end)):
            return profit_dict[(start,end)]
        
        n = len(prices[start:end])
        if n<=1:
            return 0
        
        max_sell_price = max(prices[(start+1):end])
        profit_buy_at_start = max(max_sell_price-prices[start], 0)
        profit_not_buy_at_start = self._maxProfitOneTrans(prices,start+1,end,profit_dict, max_sell_price_dict)
        profit = max(profit_buy_at_start, profit_not_buy_at_start)
        
        profit_dict[(start,end)] = profit
        max_sell_price_dict[(start,end)] = max_sell_price
        return profit    
    
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        new_prices = prices[:1]
        for i in range(1,len(prices)):
            if prices[i]>prices[i-1]:
                new_prices.append(prices[i])
            elif prices[i]<prices[i-1]:
                if i+1 > len(prices)-1:
                    new_prices.append(prices[i])
                elif prices[i]<=prices[i+1]:
                    new_prices.append(prices[i])
                
        prices = new_prices
        n = len(prices)
        
        max_profit = 0
        profit_dict = {}
        max_sell_price_dict = {}
        
        for i in range(n):
            profit = self._maxProfitOneTrans(prices,0,i,profit_dict,max_sell_price_dict) \
                   + self._maxProfitOneTrans(prices,i,n,profit_dict,max_sell_price_dict)
            if profit>max_profit:
                max_profit=profit
        return max_profit

s6 = Solution()   

In [28]:
from random import randint
prices = [randint(0,1000) for _ in range(5000)]

In [29]:
%time s1.maxProfit(prices)

CPU times: user 4.38 s, sys: 25.3 ms, total: 4.4 s
Wall time: 4.46 s


2000

In [30]:
%time s2.maxProfit(prices)

CPU times: user 9.16 s, sys: 52.7 ms, total: 9.22 s
Wall time: 9.36 s


2000

In [31]:
%time s3.maxProfit(prices)

CPU times: user 9.79 s, sys: 54.8 ms, total: 9.85 s
Wall time: 10 s


2000

In [32]:
%time s4.maxProfit(prices[:1000])

CPU times: user 9.49 s, sys: 92.2 ms, total: 9.58 s
Wall time: 9.72 s


1992

In [33]:
%time s5.maxProfit(prices[:1000])

CPU times: user 5.84 s, sys: 34.6 ms, total: 5.87 s
Wall time: 5.97 s


1992

In [34]:
%time s6.maxProfit(prices[:1000])

CPU times: user 5.16 s, sys: 92.3 ms, total: 5.25 s
Wall time: 5.35 s


1992